In [59]:
from fileinput import filename
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

import csv

def convert_pdf_to_string(file_path):
    output_string = StringIO()
    with open(file_path, 'rb') as in_file:
        parser = PDFParser(in_file)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams= LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(doc):
            interpreter.process_page(page)
        
    return(output_string.getvalue())


if __name__ == '__main__':
    originMonth = int(input("출력할 달을 입력해주세요(ex. 2월이면 2를 입력.) : "))
    pdfName = "./(주)오데야 "+ str(originMonth) +"월사업소득자.pdf"
    
    text = convert_pdf_to_string(pdfName)
    text = text.replace('.', '')
    text = text.replace('\x0c','')
    table_of_contents_raw = text.split('\n')
    #print(table_of_contents_raw)
    #for txt in table_of_contents_raw :
    #    print(txt)
    #print(convert_pdf_to_string(pdfName))
    
    name_dict = {} #이름 dictionary (ex. 0 : 첫 번째 사람 이름 ...)
    people_dict = {} #각 사람 별 정보 dictionary
    dict_index = 0
    
    #940909 #왼쪽으로 이름 찾기 #'유진'님은 두 글자라 따로 넣어 줘야됨 ㅠ
    for txt in table_of_contents_raw :
        if "940909 " in txt :
            #print(txt[7:10])
            name_dict[dict_index] = txt[7:10]
            people_dict[txt[7:10]] = []
            dict_index += 1
            
    name_dict[dict_index] = '유진'
    people_dict['유진'] = []
    
    dict_index = 0
    remove_set = {''}
    ujinFlag = True
    for name in people_dict:
        flag = False
        if name == '유진' and ujinFlag:
            ujinFlag = False
            continue
        for txt in table_of_contents_raw :
            if txt.find("동,") != -1 :
                continue
            if txt.find("길") != -1 :
                continue
            if flag :
                people_dict[name].append(txt)
            if txt == name:
                flag = True
            if flag == True and txt == '관리번호':
                flag = False
                dict_index += 1
        people_dict[name] = [p for p in people_dict[name] if p not in remove_set]
        people_dict[name] = [p for p in people_dict[name] if p.find(",") != -1]
        people_dict[name] = ' && '.join(dict.fromkeys(people_dict[name]))
        print(name)
        print(people_dict[name])
    

    


출력할 달을 입력해주세요(ex. 2월이면 2를 입력.) : 3
임진영
135,666 && 4,060 && 131,206
배소원
1,339,166 && 40,170 && 4,010 && 1,294,986
이동학
28,500
정하은
1,085,833 && 32,570 && 3,250 && 1,050,013
송지훈
49,875 && 1,490 && 48,245
홍진명
2,398,900 && 71,960 && 7,190 && 2,319,750
김혜민
1,421,833 && 42,650 && 4,260 && 1,374,923
신채영
595,000 && 17,850 && 1,780 && 575,370
최영서
171,291 && 5,130 && 165,651
김태호
306,375 && 9,190 && 296,275
박창옥
2,463,166 && 73,890 && 7,380 && 2,381,896
신지우
694,620 && 20,830 && 2,080 && 671,710
조수미
383,706 && 11,510 && 1,150 && 371,046
박인옥
1,305,831 && 39,170 && 3,910 && 1,262,751
박희자
504,666 && 15,130 && 1,510 && 488,026
이승연
27,480
최예인
364,873 && 10,940 && 1,090 && 352,843
송유진
398,460 && 11,950 && 1,190 && 385,320
조세미
122,133 && 3,660 && 118,113
정진원
102,286 && 3,060 && 98,926
이은진
54,960 && 1,640 && 53,160
김지우
27,480
차준혁
342,000 && 10,260 && 1,020 && 330,720
박영민
900,000 && 27,000 && 2,700 && 870,300
정경화
285,000 && 8,550 && 275,600
